In [8]:
import os
os.getcwd()

'/Users/izapreev/Projects/SimpDOM'

In [9]:
# Make sure the source code auto reloads into the kernel
%load_ext autoreload
%autoreload 2

In [10]:
# To help preventing shared maemory errors
!ulimit -n 500000
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [11]:
# Do the training and testing
import random
import pickle

from Utils.logger import logger
from train_and_eval import test_model
from train_and_eval import train_model
from train_and_eval import create_config

# Set training and validation sites
train_websites = ['auto-autobytel', 'auto-automotive', 'auto-autoweb', 'auto-carquotes', 'auto-cars', 'auto-kbb', 'auto-motortrend', 'auto-msn']
val_websites = ['auto-aol']
test_websites = ['auto-aol', 'auto-yahoo']

# Define the data attributes of interest
attributes = ['model', 'price', 'engine', 'fuel_economy']

# Create the model config
config = create_config(train_websites=train_websites, val_websites=val_websites, test_websites=test_websites, attributes=attributes)

# Define the number of training epochs
num_train_epochs = 10

In [12]:
logger.info(f'Starting model training')
model = train_model(config, num_train_epochs)

14:04:25 INFO (942756449:1): Starting model training
14:04:25 INFO (train_and_eval:67): Instantiating the Model checkpoint.
14:04:25 INFO (train_and_eval:77): Instantiating the Sequential model
14:04:25 INFO (SimpDOM_model:28): Instantiating the new model with config: {'out_dim': 5, 'train_websites': ['auto-autobytel', 'auto-automotive', 'auto-autoweb', 'auto-carquotes', 'auto-cars', 'auto-kbb', 'auto-motortrend', 'auto-msn'], 'val_websites': ['auto-aol'], 'test_websites': ['auto-aol', 'auto-yahoo'], 'datapath': './data', 'n_workers': 0, 'char_emb_dim': 16, 'char_hid_dim': 100, 'char_emb_dropout': 0.1, 'tag_emb_dim': 16, 'tag_hid_dim': 30, 'leaf_emb_dim': 30, 'pos_emb_dim': 20, 'attributes': ['model', 'price', 'engine', 'fuel_economy'], 'n_gpus': 0, 'class_weights': [1, 100, 100, 100, 100], 'char_dict_filename': './data/English_charDict.pkl', 'tag_dict_filename': './data/HTMLTagDict.pkl', 'word_emb_filename': './data/glove.6B.100d.txt'}
14:04:25 INFO (SimpDOM_model:79): Loading ./data/

Web sites:   0%|          | 0/1 [00:00<?, ?it/s]

Web sites:   0%|          | 0/1 [00:00<?, ?it/s]

Web sites:   0%|          | 0/1 [00:00<?, ?it/s]

Web sites:   0%|          | 0/1 [00:00<?, ?it/s]

...

15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.09728603810071945
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.14263571798801422
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.1169283539056778
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.13836325705051422
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.10570533573627472
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.14173784852027893
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.11849341541528702
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.10537511110305786
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.10982905328273773
15:34:39 INFO (SimpDOM_model:162): Mean validation loss: 0.12261694669723511
15:34:39 INFO (model_checkpoint:666): Epoch 7, global step 22168: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.0898217037320137
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.15671281516551971
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.08536389470100403
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.09976442903280258
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.07831426709890366
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.12702828645706177
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.10950875282287598
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.08828958868980408
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.08987957239151001
15:45:25 INFO (SimpDOM_model:162): Mean validation loss: 0.08516055345535278
15:45:25 INFO (model_checkpoint:666): Epoch 8, global step 24939: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.11222688108682632
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.09882201999425888
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.09507166594266891
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.09751509130001068
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.1080753356218338
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.18403437733650208
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.10048313438892365
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.1513763666152954
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.13575738668441772
15:56:31 INFO (SimpDOM_model:162): Mean validation loss: 0.12136910110712051
15:56:31 INFO (model_checkpoint:666): Epoch 9, global step 27710: 'val_loss' was not in top 1
15:56:31 INFO (fit_loop:170): `Trainer.fit` stopped: `max_epochs=10` reached.
15:56:32 INFO (train_and_eval:88): Saving the check point in

In [13]:
logger.info(f'Training is finished, starting predicting')
avg_pr_dict = test_model(config, model)
logger.info(f'Test predictions result: {avg_pr_dict}')

16:04:25 INFO (600260005:1): Training is finished, starting predicting
16:04:25 WARNING (train_and_eval:107): The test websites was not explit, extracting from config!
16:04:25 INFO (train_and_eval:112): The test websites are: ['auto-aol', 'auto-yahoo']
16:04:25 INFO (SimpDOM_model:182): Creating testing data loader for: ['auto-aol', 'auto-yahoo']
16:04:25 INFO (swde_dataLoader:257): Start loading data set for websites: ['auto-aol', 'auto-yahoo']


Web site:   0%|          | 0/2 [00:00<?, ?it/s]

Web pages for: "auto-aol":   0%|          | 0/2000 [00:00<?, ?it/s]

Web pages for: "auto-yahoo":   0%|          | 0/2000 [00:00<?, ?it/s]

16:04:44 INFO (swde_dataLoader:262): SWDE data test loader has loaded: 435018 nodes from: ['auto-aol', 'auto-yahoo']


Testing batches:   0%|          | 0/13595 [00:00<?, ?it/s]

Dumping predictions dataframe into: test_predictions.csv


16:20:01 INFO (train_and_eval:122): Prediction summary:
{1: (0.7898609975470156, 0.48360450563204005, 0.599906846762925), 2: (0.98975, 0.98975, 0.98975), 3: (0.9940443963183541, 0.836, 0.9081977650312476), 4: (0.9687108886107635, 0.9958387516254876, 0.9820875199510357)}


class - 1: precision = 0.7898609975470156, recall = 0.48360450563204005, F1 = 0.599906846762925
class - 2: precision = 0.98975, recall = 0.98975, F1 = 0.98975
class - 3: precision = 0.9940443963183541, recall = 0.836, F1 = 0.9081977650312476
class - 4: precision = 0.9687108886107635, recall = 0.9958387516254876, F1 = 0.9820875199510357


16:20:14 INFO (train_and_eval:126): Website-level prediction summary:
      website  attribute  precision    recall  retrieved_relevant  retrieved  \
0    auto-aol          1   0.922923  0.924311                1844       1998   
1    auto-aol          2   0.998000  0.998000                1996       2000   
2    auto-aol          3   0.000000  0.000000                   0          0   
3    auto-aol          4   0.936029  0.991328                1829       1954   
4  auto-yahoo          1   0.196429  0.044000                  88        448   
5  auto-yahoo          2   0.981500  0.981500                1963       2000   
6  auto-yahoo          3   0.987013  0.836000                1672       1694   
7  auto-yahoo          4   1.000000  1.000000                2000       2000   

   relevant  
0      1995  
1      2000  
2         0  
3      1845  
4      2000  
5      2000  
6      2000  
7      2000  
16:20:14 INFO (600260005:3): Test predictions result: {1: (0.7898609975470156, 0.48

In [ ]:
# Check the accuracy on the data used for training
avg_pr_dict = test_model(config, model, test_websites=val_websites)
logger.info(f'Test predictions result: {avg_pr_dict}')

16:22:51 WARNING (train_and_eval:110): The test websites was explit, updating config!
16:22:51 INFO (train_and_eval:112): The test websites are: ['auto-aol']
16:22:51 INFO (SimpDOM_model:182): Creating testing data loader for: ['auto-aol', 'auto-yahoo']
16:22:51 INFO (swde_dataLoader:257): Start loading data set for websites: ['auto-aol', 'auto-yahoo']


Web site:   0%|          | 0/2 [00:00<?, ?it/s]

Web pages for: "auto-aol":   0%|          | 0/2000 [00:00<?, ?it/s]

Web pages for: "auto-yahoo":   0%|          | 0/2000 [00:00<?, ?it/s]

16:23:13 INFO (swde_dataLoader:262): SWDE data test loader has loaded: 435018 nodes from: ['auto-aol', 'auto-yahoo']


Testing batches:   0%|          | 0/13595 [00:00<?, ?it/s]

In [ ]:
# To Create the training data for other verticals.

In [3]:
Datapath = './data' #directory path where each vertical's html pages from SWDE dataset (website wise folders)
vertical = 'auto'

In [4]:
#dataset processor: to find the set of fixed nodes for each of the websites
from DatasetCreation.storeFixedNodes import main as storeFixedNodes
storeFixedNodes(Datapath, vertical)

Web sites:   0%|          | 0/10 [00:00<?, ?it/s]

Pages for website: auto-kbb:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for website: auto-autoweb:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for website: auto-aol:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for website: auto-yahoo:   0%|          | 0/2000 [00:00<?, ?it/s]

...

Pages for website: auto-carquotes:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for website: auto-cars:   0%|          | 0/657 [00:00<?, ?it/s]

Pages for website: auto-msn:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for website: auto-automotive:   0%|          | 0/1999 [00:00<?, ?it/s]

13:26:59 INFO (storeFixedNodes:60): Got: 1973 fixed nodes for: auto-automotive, dumping into: ./data/fixedNodes_camera.csv


In [5]:
#dataset processor: Store each website as pickle file
from DatasetCreation.storeAllTextNodes import main as storeAllTextNodes
storeAllTextNodes(Datapath, vertical)

Web sites:   0%|          | 0/10 [00:00<?, ?it/s]

Pages for website: auto-kbb:   0%|          | 0/2000 [00:00<?, ?it/s]

13:27:16 INFO (storeAllTextNodes:53): Dumping node details into: ./data/nodesDetails/auto-kbb.pkl
13:27:17 INFO (storeAllTextNodes:57): The average variabe/mixed node counts: auto-kbb are: [113.325  149.8585]


Pages for website: auto-autoweb:   0%|          | 0/2000 [00:00<?, ?it/s]

13:27:29 INFO (storeAllTextNodes:53): Dumping node details into: ./data/nodesDetails/auto-autoweb.pkl
13:27:29 INFO (storeAllTextNodes:57): The average variabe/mixed node counts: auto-autoweb are: [ 42.697  117.3055]


...

13:28:58 INFO (storeAllTextNodes:53): Dumping node details into: ./data/nodesDetails/auto-cars.pkl
13:28:58 INFO (storeAllTextNodes:57): The average variabe/mixed node counts: auto-cars are: [52.79604262 90.49467275]


Pages for website: auto-msn:   0%|          | 0/2000 [00:00<?, ?it/s]

13:29:33 INFO (storeAllTextNodes:53): Dumping node details into: ./data/nodesDetails/auto-msn.pkl
13:29:35 INFO (storeAllTextNodes:57): The average variabe/mixed node counts: auto-msn are: [ 95.1715 283.4425]


Pages for website: auto-automotive:   0%|          | 0/1999 [00:00<?, ?it/s]

13:30:14 INFO (storeAllTextNodes:53): Dumping node details into: ./data/nodesDetails/auto-automotive.pkl
13:30:15 INFO (storeAllTextNodes:57): The average variabe/mixed node counts: auto-automotive are: [ 77.32616308 298.83491746]


In [6]:
#dataset processor: append the friendCircle of each node in the DOMTree
from Utils.friendCircleExtractioin import main as friendCircleExtractioin
friendCircleExtractioin(Datapath, vertical, 5)

Web sites:   0%|          | 0/10 [00:00<?, ?it/s]

Pages for website: auto-kbb:   0%|          | 0/2000 [00:00<?, ?it/s]

13:30:24 INFO (friendCircleExtractioin:99): Re-dumping node details (all pages) into: ./data/nodesDetails/auto-kbb.pkl
13:30:26 INFO (friendCircleExtractioin:102): The friend/partner nodes for: "auto-kbb" is: 9.955104522341177/0.11822890216473102


Pages for website: auto-autoweb:   0%|          | 0/2000 [00:00<?, ?it/s]

13:30:31 INFO (friendCircleExtractioin:99): Re-dumping node details (all pages) into: ./data/nodesDetails/auto-autoweb.pkl
13:30:32 INFO (friendCircleExtractioin:102): The friend/partner nodes for: "auto-autoweb" is: 9.929001578443978/0.8581395790461918


...

13:31:11 INFO (friendCircleExtractioin:99): Re-dumping node details (all pages) into: ./data/nodesDetails/auto-cars.pkl
13:31:11 INFO (friendCircleExtractioin:102): The friend/partner nodes for: "auto-cars" is: 10.0/0.5597516234882789


Pages for website: auto-msn:   0%|          | 0/2000 [00:00<?, ?it/s]

13:31:22 INFO (friendCircleExtractioin:99): Re-dumping node details (all pages) into: ./data/nodesDetails/auto-msn.pkl
13:31:24 INFO (friendCircleExtractioin:102): The friend/partner nodes for: "auto-msn" is: 9.989908327451145/0.29792965069489014


Pages for website: auto-automotive:   0%|          | 0/1999 [00:00<?, ?it/s]

13:31:34 INFO (friendCircleExtractioin:99): Re-dumping node details (all pages) into: ./data/nodesDetails/auto-automotive.pkl
13:31:36 INFO (friendCircleExtractioin:102): The friend/partner nodes for: "auto-automotive" is: 10.0/0.47887502249239666


In [7]:
#dataset processor: associate the ground truth label to each node
attributes = ['model', 'price', 'engine', 'fuel_economy']
from DatasetCreation.assignGroundTruthToEachNode import main as assignGroundTruthToEachNode
assignGroundTruthToEachNode(Datapath, vertical, attributes)

Web sites::   0%|          | 0/10 [00:00<?, ?it/s]

Pages for attribute: model:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for attribute: price:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for attribute: engine:   0%|          | 0/2000 [00:00<?, ?it/s]

Pages for attribute: fuel_economy:   0%|          | 0/2000 [00:00<?, ?it/s]

...

Pages for attribute: model:   0%|          | 0/1999 [00:00<?, ?it/s]

Pages for attribute: price:   0%|          | 0/1999 [00:00<?, ?it/s]

Pages for attribute: engine:   0%|          | 0/1999 [00:00<?, ?it/s]

Pages for attribute: fuel_economy:   0%|          | 0/1999 [00:00<?, ?it/s]

13:34:35 INFO (assignGroundTruthToEachNode:73): Re-dumping node details (all pages) into: ./data/nodesDetails/auto-automotive.pkl
